In [4]:
import pickle
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [5]:
data = pd.read_csv("data/heart_disease.csv")
data.head()

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
0,63,1,asymptomatic,145,233.0,1,0,150.0,0,2.3,0,0,1,1
1,37,1,non-anginal pain,130,250.0,0,1,187.0,0,3.5,0,0,2,1
2,41,0,atypical angina,130,204.0,0,0,172.0,0,1.4,2,0,2,1
3,56,1,atypical angina,120,236.0,0,1,178.0,0,0.8,2,0,2,1
4,57,0,typical anginia,120,354.0,0,1,163.0,1,0.6,2,0,2,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 14 columns):
age                        366 non-null int64
sex                        366 non-null int64
chest_pain_type            366 non-null object
resting_blood_pressure     366 non-null int64
cholesterol                348 non-null float64
fasting_blood_sugar        366 non-null int64
rest_ecg                   366 non-null int64
max_heart_rate_achieved    339 non-null float64
exercise_induced_angina    366 non-null int64
st_depression              366 non-null float64
st_slope                   366 non-null int64
num_major_vessels          366 non-null int64
thalassemia                366 non-null int64
target                     366 non-null int64
dtypes: float64(3), int64(10), object(1)
memory usage: 40.2+ KB


In [7]:
# Checking for Missing values
print("Any NULL values present?")
print(data.isnull().sum())

Any NULL values present?
age                         0
sex                         0
chest_pain_type             0
resting_blood_pressure      0
cholesterol                18
fasting_blood_sugar         0
rest_ecg                    0
max_heart_rate_achieved    27
exercise_induced_angina     0
st_depression               0
st_slope                    0
num_major_vessels           0
thalassemia                 0
target                      0
dtype: int64


In [8]:
print("Checking the unique values in Cholesterol column : \n", data["cholesterol"].unique())
print("Checking the unique values in max_heart_rate column : \n", data["max_heart_rate_achieved"].unique())

Checking the unique values in Cholesterol column : 
 [233. 250. 204. 236. 354. 192. 294. 263. 199. 168. 239. 275. 266. 211.
 283. 219. 340. 226. 247. 234. 243. 302. 212. 175. 417. 197. 198. 177.
 273. 213. 304. 232.  nan 360. 308. 245. 208. 264. 321. 257. 216. 256.
 231. 141. 252. 222. 260. 182. 303. 265. 309. 186. 203. 183. 220. 209.
 258. 227. 261. 221. 205. 240. 318. 298. 564. 277. 214. 248. 255. 207.
 223. 288. 160. 394. 315. 246. 244. 270. 195. 196. 254. 325. 126. 313.
 262. 215. 193. 271. 268. 267. 269. 201. 295. 235. 306. 178. 242. 180.
 228. 149. 278. 253. 342. 286. 229. 284. 224. 206. 167. 230. 335. 353.
 225. 330. 290. 172. 305. 188. 282. 185. 326. 274. 307. 249. 341. 407.
 174. 281. 289. 322. 299. 300. 184. 409. 259. 200. 327. 237. 218. 319.
 166. 311. 169. 187. 176. 241. 131.]
Checking the unique values in max_heart_rate column : 
 [150. 187. 172. 178. 163. 148. 153. 173. 162. 174. 160. 139. 171. 144.
 158. 114. 151. 161. 179. 137. 157. 123. 152.  nan 140. 188. 125. 170.
 1

In [9]:
# Replacing the missing column values with the Median
data['cholesterol'] = data['cholesterol'].replace(np.nan, data['cholesterol'].median())
data['max_heart_rate_achieved'] = data['max_heart_rate_achieved'].replace(np.nan, data['max_heart_rate_achieved'].median())

In [10]:
# Checking for Missing values after replacing values
print("Any NULL values present?")
print(data.isnull().sum())

Any NULL values present?
age                        0
sex                        0
chest_pain_type            0
resting_blood_pressure     0
cholesterol                0
fasting_blood_sugar        0
rest_ecg                   0
max_heart_rate_achieved    0
exercise_induced_angina    0
st_depression              0
st_slope                   0
num_major_vessels          0
thalassemia                0
target                     0
dtype: int64


In [11]:
# Checking the shape of the data
print("Shape of the data : ", data.shape)

Shape of the data :  (366, 14)


In [12]:
print("Any Duplicate rows present? : ", data.duplicated().any())
print("Total No.of Duplicate rows : ", data.duplicated().sum())

Any Duplicate rows present? :  True
Total No.of Duplicate rows :  41


In [13]:
data.drop_duplicates(inplace = True)
print("Shape of the data after removing duplicates : ", data.shape)

Shape of the data after removing duplicates :  (325, 14)


In [14]:
# Checking for balance/imbalance data (Balanced, if class ratio has 80:20 minimum)
data["target"].value_counts()

1    174
0    151
Name: target, dtype: int64

In [15]:
# Handling chest pain type column
print("Checking the unique values in Chest pain type column : \n", data["chest_pain_type"].unique())

Checking the unique values in Chest pain type column : 
 ['asymptomatic' 'non-anginal pain' 'atypical angina' 'typical anginia'
 'typical angina']


In [16]:
# Handling typo 'typical anginia' in the chest pain type column
data['chest_pain_type'] = data['chest_pain_type'].str.replace(pat = 'typical anginia', repl = 'typical angina')

In [17]:
print("Checking the unique values in Chest pain type column after correction : \n", data["chest_pain_type"].unique())

Checking the unique values in Chest pain type column after correction : 
 ['asymptomatic' 'non-anginal pain' 'atypical angina' 'typical angina']


In [18]:
# Performing the dummification of the chest pain type column
data = pd.get_dummies(data = data, columns = ['chest_pain_type'], drop_first = True)
data.head()

,age,sex,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target,chest_pain_type_atypical angina,chest_pain_type_non-anginal pain,chest_pain_type_typical angina
0,63,1,145,233.0,1,0,150.0,0,2.3,0,0,1,1,0,0,0
1,37,1,130,250.0,0,1,187.0,0,3.5,0,0,2,1,0,1,0
2,41,0,130,204.0,0,0,172.0,0,1.4,2,0,2,1,1,0,0
3,56,1,120,236.0,0,1,178.0,0,0.8,2,0,2,1,1,0,0
4,57,0,120,354.0,0,1,163.0,1,0.6,2,0,2,1,0,0,1


In [19]:
X = data.loc[:,data.columns != 'target']
X.head()

,age,sex,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,chest_pain_type_atypical angina,chest_pain_type_non-anginal pain,chest_pain_type_typical angina
0,63,1,145,233.0,1,0,150.0,0,2.3,0,0,1,0,0,0
1,37,1,130,250.0,0,1,187.0,0,3.5,0,0,2,0,1,0
2,41,0,130,204.0,0,0,172.0,0,1.4,2,0,2,1,0,0
3,56,1,120,236.0,0,1,178.0,0,0.8,2,0,2,1,0,0
4,57,0,120,354.0,0,1,163.0,1,0.6,2,0,2,0,0,1


In [20]:
Y = data['target']
Y.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, stratify = Y)
print('Training Data Shape:', X_train.shape, Y_train.shape)
print('Testing Data Shape:', X_test.shape, Y_test.shape)

Training Data Shape: (260, 15) (260,)
Testing Data Shape: (65, 15) (65,)


In [31]:
# Performing standard scaling of independent features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

array([[-1.53766127,  0.69084928, -0.10171371, ..., -0.44514793,
         1.66376559, -1.00772212],
       [-1.76107112,  0.69084928, -0.77405855, ..., -0.44514793,
        -0.60104621,  0.99233705],
       [ 0.24961755,  0.69084928,  0.45857366, ..., -0.44514793,
        -0.60104621,  0.99233705],
       ...,
       [ 1.47837173,  0.69084928,  0.68268861, ..., -0.44514793,
        -0.60104621,  0.99233705],
       [ 1.25496188, -1.44749373,  0.79474608, ..., -0.44514793,
         1.66376559, -1.00772212],
       [ 0.36132247,  0.69084928,  0.79474608, ..., -0.44514793,
        -0.60104621,  0.99233705]])

In [23]:
param_dist = [
    {'C': (np.arange(0.1,1,0.1)) , 'kernel': ['linear']},
    {'C': (np.arange(0.1,1.1,0.1)) , 'gamma': ['auto','auto_deprecated','scale'], 'kernel': ['rbf']},
    {'degree' : [2,3,4,5,6] ,'gamma' :[0.01,0.02,0.03,0.04,0.05], 'C' :(np.arange(0.1,1,0.1)) , 'kernel':['poly']}
]

print(param_dist)

[{'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), 'kernel': ['linear']}, {'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]), 'gamma': ['auto', 'auto_deprecated', 'scale'], 'kernel': ['rbf']}, {'degree': [2, 3, 4, 5, 6], 'gamma': [0.01, 0.02, 0.03, 0.04, 0.05], 'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), 'kernel': ['poly']}]


In [24]:
# Hyperparameter tuning of the SVM model
svm_model = SVC()
model_svm = GridSearchCV(svm_model,
                         param_grid = param_dist,
                         n_jobs = 1,
                         scoring = 'f1',
                         cv = 10,                         
                         verbose = 2)

model_svm.fit(X_train, Y_train)

Fitting 10 folds for each of 264 candidates, totalling 2640 fits
[CV] C=0.1, kernel=linear ............................................
[CV] ............................. C=0.1, kernel=linear, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ............................. C=0.1, kernel=linear, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ............................. C=0.1, kernel=linear, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ............................. C=0.1, kernel=linear, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ............................. C=0.1, kernel=linear, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ............................. C=0.1, kernel=linear, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] .......

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ............................. C=0.4, kernel=linear, total=   0.0s
[CV] C=0.4, kernel=linear ............................................
[CV] ............................. C=0.4, kernel=linear, total=   0.0s
[CV] C=0.4, kernel=linear ............................................
[CV] ............................. C=0.4, kernel=linear, total=   0.0s
[CV] C=0.4, kernel=linear ............................................
[CV] ............................. C=0.4, kernel=linear, total=   0.0s
[CV] C=0.4, kernel=linear ............................................
[CV] ............................. C=0.4, kernel=linear, total=   0.0s
[CV] C=0.4, kernel=linear ............................................
[CV] ............................. C=0.4, kernel=linear, total=   0.0s
[CV] C=0.4, kernel=linear ............................................
[CV] ............................. C=0.4, kernel=linear, total=   0.0s
[CV] C=0.4, kernel=linear ............................................
[CV] .

[CV] C=0.1, gamma=auto, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=auto, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=auto, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=auto, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=auto, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=auto, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=auto, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=auto, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=auto, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=auto, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=auto, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=auto, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=auto_deprecated, kernel=rbf ........................
[CV] ......... C=0.1, gamma=auto_deprecated, kernel=rbf, total=   0.0s
[CV] C

C:\Users\vaithy\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\vaithy\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\vaithy\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\vaithy\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureW

...

[CV] ......... C=0.9, degree=2, gamma=0.05, kernel=poly, total=   0.0s
[CV] C=0.9, degree=2, gamma=0.05, kernel=poly ........................
[CV] ......... C=0.9, degree=2, gamma=0.05, kernel=poly, total=   0.0s
[CV] C=0.9, degree=2, gamma=0.05, kernel=poly ........................
[CV] ......... C=0.9, degree=2, gamma=0.05, kernel=poly, total=   0.0s
[CV] C=0.9, degree=2, gamma=0.05, kernel=poly ........................
[CV] ......... C=0.9, degree=2, gamma=0.05, kernel=poly, total=   0.0s
[CV] C=0.9, degree=2, gamma=0.05, kernel=poly ........................
[CV] ......... C=0.9, degree=2, gamma=0.05, kernel=poly, total=   0.0s
[CV] C=0.9, degree=2, gamma=0.05, kernel=poly ........................
[CV] ......... C=0.9, degree=2, gamma=0.05, kernel=poly, total=   0.0s
[CV] C=0.9, degree=2, gamma=0.05, kernel=poly ........................
[CV] ......... C=0.9, degree=2, gamma=0.05, kernel=poly, total=   0.0s
[CV] C=0.9, degree=3, gamma=0.01, kernel=poly ........................
[CV] .

[CV] ......... C=0.9, degree=4, gamma=0.02, kernel=poly, total=   0.0s
[CV] C=0.9, degree=4, gamma=0.02, kernel=poly ........................
[CV] ......... C=0.9, degree=4, gamma=0.02, kernel=poly, total=   0.0s
[CV] C=0.9, degree=4, gamma=0.02, kernel=poly ........................
[CV] ......... C=0.9, degree=4, gamma=0.02, kernel=poly, total=   0.0s
[CV] C=0.9, degree=4, gamma=0.02, kernel=poly ........................
[CV] ......... C=0.9, degree=4, gamma=0.02, kernel=poly, total=   0.0s
[CV] C=0.9, degree=4, gamma=0.02, kernel=poly ........................
[CV] ......... C=0.9, degree=4, gamma=0.02, kernel=poly, total=   0.0s
[CV] C=0.9, degree=4, gamma=0.02, kernel=poly ........................
[CV] ......... C=0.9, degree=4, gamma=0.02, kernel=poly, total=   0.0s
[CV] C=0.9, degree=4, gamma=0.02, kernel=poly ........................
[CV] ......... C=0.9, degree=4, gamma=0.02, kernel=poly, total=   0.0s
[CV] C=0.9, degree=4, gamma=0.03, kernel=poly ........................
[CV] .

[CV] C=0.9, degree=5, gamma=0.03, kernel=poly ........................
[CV] ......... C=0.9, degree=5, gamma=0.03, kernel=poly, total=   0.0s
[CV] C=0.9, degree=5, gamma=0.04, kernel=poly ........................
[CV] ......... C=0.9, degree=5, gamma=0.04, kernel=poly, total=   0.0s
[CV] C=0.9, degree=5, gamma=0.04, kernel=poly ........................
[CV] ......... C=0.9, degree=5, gamma=0.04, kernel=poly, total=   0.0s
[CV] C=0.9, degree=5, gamma=0.04, kernel=poly ........................
[CV] ......... C=0.9, degree=5, gamma=0.04, kernel=poly, total=   0.0s
[CV] C=0.9, degree=5, gamma=0.04, kernel=poly ........................
[CV] ......... C=0.9, degree=5, gamma=0.04, kernel=poly, total=   0.0s
[CV] C=0.9, degree=5, gamma=0.04, kernel=poly ........................
[CV] ......... C=0.9, degree=5, gamma=0.04, kernel=poly, total=   0.0s
[CV] C=0.9, degree=5, gamma=0.04, kernel=poly ........................
[CV] ......... C=0.9, degree=5, gamma=0.04, kernel=poly, total=   0.0s
[CV] C

[Parallel(n_jobs=1)]: Done 2640 out of 2640 | elapsed:   14.3s finished
C:\Users\vaithy\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=1,
             param_grid=[{'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,...0.9]),
                          'kernel': ['linear']},
                         {'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                          'gamma': ['auto', 'auto_deprecated', 'scale'],
                          'kernel': ['rbf']},
                         {'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                          'degree': [2, 3, 4, 5, 6],
                          'gamma': [0.01, 0.02, 0.03, 0.04, 0.05],
     

In [25]:
model_svm.best_params_

{'C': 0.5, 'kernel': 'linear'}

In [26]:
svm = SVC(C = 0.5, kernel = 'linear')
svm.fit(X_train, Y_train)

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [27]:
Y_pred = svm.predict(X_test)
print("Accuracy score : ", accuracy_score(Y_test, Y_pred))
print("Precision score : ", precision_score(Y_test, Y_pred))
print("Recall score : ", recall_score(Y_test, Y_pred))
print("F1 - score : ", f1_score(Y_test, Y_pred))

Accuracy score :  0.8923076923076924
Precision score :  0.868421052631579
Recall score :  0.9428571428571428
F1 - score :  0.904109589041096


In [28]:
# Building the Pipeline

model = Pipeline([('scaler', StandardScaler()), ('svc', SVC(C = 0.5, kernel = 'linear'))])
model.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='linear', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [29]:
Y_pred = model.predict(X_test)
print("Accuracy score : ", accuracy_score(Y_test, Y_pred))
print("Precision score : ", precision_score(Y_test, Y_pred))
print("Recall score : ", recall_score(Y_test, Y_pred))
print("F1 - score : ", f1_score(Y_test, Y_pred))

Accuracy score :  0.8923076923076924
Precision score :  0.868421052631579
Recall score :  0.9428571428571428
F1 - score :  0.904109589041096


In [30]:
# open a file, where you ant to store the data
file = open('cardiology-model.pkl', 'wb')

# dump information to the file
pickle.dump(model, file)